In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import cv2
import math
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
import torch

##yolo

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 15578, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 15578 (delta 81), reused 114 (delta 58), pack-reused 15393
Receiving objects: 100% (15578/15578), 14.64 MiB | 17.17 MiB/s, done.
Resolving deltas: 100% (10605/10605), done.
/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00


## Data preprocessing

In [ ]:
output_dir = "/content/drive/MyDrive/DeepLearning/Final/Train_proc"

In [ ]:
# Define the input and output directories
input_dir = "/content/drive/MyDrive/DeepLearning/Final/Train"
output_dir = "/content/drive/MyDrive/DeepLearning/Final/Train_proc"

# Define the desired video size, length, and max_frames
video_size = (320, 320)
desired_length = 10  # 10 seconds
fps = 30  # Assuming 30 frames per second
max_frames = desired_length * fps

In [ ]:
# Loop over the action folders and preprocess the videos
video_data = []
video_labels = []

for label, action in enumerate(["running", "hugging", "dancing"]):
    action_dir = os.path.join(input_dir, action)

    # Loop over the video files and preprocess each one
    for video_file in os.listdir(action_dir):
        video_path = os.path.join(action_dir, video_file)

        # Load the video and resize the frames
        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        step = total_frames // (desired_length * fps)
        step = max(step, 1)  # Ensure step is never zero

        frames = []
        count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            if count % step == 0:
                frame = cv2.resize(frame, video_size)
                frames.append(frame)

                if len(frames) == desired_length * fps:
                    break
            count += 1
        cap.release()
            
        # Pad or truncate the frames
        if len(frames) < max_frames:
            # Pad with the last frame if the video is shorter than max_frames
            padding = [frames[-1]] * (max_frames - len(frames))
            frames = frames + padding
        elif len(frames) > max_frames:
            # Truncate if the video is longer than max_frames
            frames = frames[:max_frames]

        # Append the preprocessed frames and labels to the lists
        video_data.append(np.array(frames))
        video_labels.append(np.array([label], dtype=int))

# Convert the lists to numpy arrays
video_data = np.array(video_data)
video_labels = np.array(video_labels)

# Save the preprocessed video data to numpy files
np.save(os.path.join(output_dir, "video_data.npy"), video_data)
np.save(os.path.join(output_dir, "video_labels.npy"), video_labels)


In [ ]:
# Load the preprocessed video data
video_data = np.load(os.path.join(output_dir, "video_data.npy"), allow_pickle=True)
video_labels = np.load(os.path.join(output_dir, "video_labels.npy"), allow_pickle=True)

# # Normalize the video_data
# # video_data = video_data.astype('float32')

print(video_data.shape)
print(video_labels.shape)

(195, 300, 320, 320, 3)
(195, 1)


In [ ]:
images_folder = '/content/drive/MyDrive/DeepLearning/Final/hundred_frames'
os.makedirs(images_folder, exist_ok=True)
for i, video in enumerate(video_data):
    hundred_frame = video[99]
    resized_frame = cv2.resize(hundred_frame, (640, 640))
    image_path = os.path.join(images_folder, f"hundred_frame_{i}.png")
    cv2.imwrite(image_path, cv2.cvtColor(resized_frame, cv2.COLOR_RGB2BGR))

In [ ]:
# Model
model = torch.hub.load("ultralytics/yolov5", "yolov5x")  # or yolov5n - yolov5x6, custom

/usr/local/lib/python3.9/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-4-22 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)



requirements: /root/.cache/torch/hub/requirements.txt not found, check failed.


100%|██████████| 166M/166M [00:09<00:00, 17.7MB/s]

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
# Preprocess videos
def preprocess_video(video):
    # Resize video frames to the expected input size of YOLOv5
    resized_video = []
    for frame in video:
        resized_frame = cv2.resize(frame, (640, 640))
        resized_video.append(resized_frame)
    return np.array(resized_video)

# # Process all videos
# preprocessed_videos = [preprocess_video(video_data[i]) for i in range(video_data.shape[0])]

In [ ]:
# Create a directory to store preprocessed videos
os.makedirs("/content/drive/MyDrive/DeepLearning/Final/one_by_one", exist_ok=True)

# Save preprocessed videos to disk
for i in range(video_data.shape[0]):
    preprocessed_video = preprocess_video(video_data[i])
    np.save(f"/content/drive/MyDrive/DeepLearning/Final/one_by_one/video_{i}.npy", preprocessed_video)

In [ ]:
def npy_to_mp4(npy_path, output_dir):
    # Load the numpy array
    video_data = np.load(npy_path)

    # Define the video file name
    video_name = os.path.splitext(os.path.basename(npy_path))[0] + '.mp4'
    output_path = os.path.join(output_dir, video_name)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    height, width, _ = video_data[0].shape
    out = cv2.VideoWriter(output_path, fourcc, 30.0, (width, height))

    # Write each frame to the video file
    for frame in video_data:
        out.write(frame)

    # Release the VideoWriter
    out.release()

In [ ]:
input_npy_dir = '/content/drive/MyDrive/DeepLearning/Final/one_by_one'
output_video_dir = '/content/drive/MyDrive/DeepLearning/Final/ten_video'

os.makedirs(output_video_dir, exist_ok=True)


# List of specific files to convert
specific_files = [
    "video_1.npy", "video_12.npy", "video_32.npy", "video_36.npy", "video_80.npy",
    "video_181.npy", "video_193.npy", "video_136.npy", "video_192.npy", "video_154.npy"
]

# Process only the specified .npy files
for npy_file in specific_files:
    npy_path = os.path.join(input_npy_dir, npy_file)
    if os.path.exists(npy_path):
        npy_to_mp4(npy_path, output_video_dir)
    else:
        print(f"{npy_file} not found in {input_npy_dir}")


In [ ]:
!python train.py --img 640 --batch 5 --epochs 200 --data dl_data.yaml --weights yolov5s.pt --nosave

train: weights=yolov5s.pt, cfg=, data=dl_data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=200, batch_size=5, imgsz=640, rect=False, resume=False, nosave=True, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-151-g3e14883 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2,

In [ ]:
!python detect.py --source '/content/drive/MyDrive/DeepLearning/Final/ten_video/video_1.mp4' --weights runs/train/exp4/weights/last.pt --conf 0.25 --img-size 640 --save-txt --project '/content/drive/MyDrive/DeepLearning/Final/fine_tune_output' --name 'output_1'


detect: weights=['runs/train/exp4/weights/last.pt'], source=/content/drive/MyDrive/DeepLearning/Final/ten_video/video_1.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/DeepLearning/Final/fine_tune_output, name=output_1, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-150-gbe61a64 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/300) /content/drive/.shortcut-targets-by-id/1S_QEvwuA5fxYIBSwSiE3L8oOvREzRGi8/Final/ten_video/video_1.mp4: 640x640 1 person, 11.5ms
video 1/1 (2/300) /content/drive/.shortcu

In [ ]:
!python detect.py --source '/content/drive/MyDrive/DeepLearning/Final/ten_video/video_12.mp4' --weights runs/train/exp4/weights/last.pt --conf 0.25 --img-size 640 --save-txt --project '/content/drive/MyDrive/DeepLearning/Final/fine_tune_output' --name 'output_12'


detect: weights=['runs/train/exp4/weights/last.pt'], source=/content/drive/MyDrive/DeepLearning/Final/ten_video/video_12.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/DeepLearning/Final/fine_tune_output, name=output_12, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-150-gbe61a64 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/300) /content/drive/.shortcut-targets-by-id/1S_QEvwuA5fxYIBSwSiE3L8oOvREzRGi8/Final/ten_video/video_12.mp4: 640x640 5 persons, 11.6ms
video 1/1 (2/300) /content/drive/.sho

In [ ]:
!python detect.py --source '/content/drive/MyDrive/DeepLearning/Final/ten_video/video_136.mp4' --weights runs/train/exp4/weights/last.pt --conf 0.25 --img-size 640 --save-txt --project '/content/drive/MyDrive/DeepLearning/Final/fine_tune_output' --name 'output_136'


detect: weights=['runs/train/exp4/weights/last.pt'], source=/content/drive/MyDrive/DeepLearning/Final/ten_video/video_136.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/DeepLearning/Final/fine_tune_output, name=output_136, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-150-gbe61a64 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/300) /content/drive/.shortcut-targets-by-id/1S_QEvwuA5fxYIBSwSiE3L8oOvREzRGi8/Final/ten_video/video_136.mp4: 640x640 2 persons, 11.6ms
video 1/1 (2/300) /content/drive/.

In [ ]:
!python detect.py --source '/content/drive/MyDrive/DeepLearning/Final/ten_video/video_154.mp4' --weights runs/train/exp4/weights/last.pt --conf 0.25 --img-size 640 --save-txt --project '/content/drive/MyDrive/DeepLearning/Final/fine_tune_output' --name 'output_154'


detect: weights=['runs/train/exp4/weights/last.pt'], source=/content/drive/MyDrive/DeepLearning/Final/ten_video/video_154.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/DeepLearning/Final/fine_tune_output, name=output_154, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-150-gbe61a64 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/300) /content/drive/.shortcut-targets-by-id/1S_QEvwuA5fxYIBSwSiE3L8oOvREzRGi8/Final/ten_video/video_154.mp4: 640x640 1 person, 11.6ms
video 1/1 (2/300) /content/drive/.s

In [ ]:
!python detect.py --source '/content/drive/MyDrive/DeepLearning/Final/ten_video/video_181.mp4' --weights runs/train/exp4/weights/last.pt --conf 0.25 --img-size 640 --save-txt --project '/content/drive/MyDrive/DeepLearning/Final/fine_tune_output' --name 'output_181'


detect: weights=['runs/train/exp4/weights/last.pt'], source=/content/drive/MyDrive/DeepLearning/Final/ten_video/video_181.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/DeepLearning/Final/fine_tune_output, name=output_181, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-150-gbe61a64 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/300) /content/drive/.shortcut-targets-by-id/1S_QEvwuA5fxYIBSwSiE3L8oOvREzRGi8/Final/ten_video/video_181.mp4: 640x640 1 person, 11.6ms
video 1/1 (2/300) /content/drive/.s

In [ ]:
!python detect.py --source '/content/drive/MyDrive/DeepLearning/Final/ten_video/video_192.mp4' --weights runs/train/exp4/weights/last.pt --conf 0.25 --img-size 640 --save-txt --project '/content/drive/MyDrive/DeepLearning/Final/fine_tune_output' --name 'output_192'


detect: weights=['runs/train/exp4/weights/last.pt'], source=/content/drive/MyDrive/DeepLearning/Final/ten_video/video_192.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/DeepLearning/Final/fine_tune_output, name=output_192, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-150-gbe61a64 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/300) /content/drive/.shortcut-targets-by-id/1S_QEvwuA5fxYIBSwSiE3L8oOvREzRGi8/Final/ten_video/video_192.mp4: 640x640 3 persons, 11.6ms
video 1/1 (2/300) /content/drive/.

In [ ]:
!python detect.py --source '/content/drive/MyDrive/DeepLearning/Final/ten_video/video_193.mp4' --weights runs/train/exp4/weights/last.pt --conf 0.25 --img-size 640 --save-txt --project '/content/drive/MyDrive/DeepLearning/Final/fine_tune_output' --name 'output_193'


detect: weights=['runs/train/exp4/weights/last.pt'], source=/content/drive/MyDrive/DeepLearning/Final/ten_video/video_193.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/DeepLearning/Final/fine_tune_output, name=output_193, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-150-gbe61a64 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/300) /content/drive/.shortcut-targets-by-id/1S_QEvwuA5fxYIBSwSiE3L8oOvREzRGi8/Final/ten_video/video_193.mp4: 640x640 10 persons, 11.6ms
video 1/1 (2/300) /content/drive/

In [ ]:
!python detect.py --source '/content/drive/MyDrive/DeepLearning/Final/ten_video/video_32.mp4' --weights runs/train/exp4/weights/last.pt --conf 0.25 --img-size 640 --save-txt --project '/content/drive/MyDrive/DeepLearning/Final/fine_tune_output' --name 'output_32'


detect: weights=['runs/train/exp4/weights/last.pt'], source=/content/drive/MyDrive/DeepLearning/Final/ten_video/video_32.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/DeepLearning/Final/fine_tune_output, name=output_32, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-150-gbe61a64 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/300) /content/drive/.shortcut-targets-by-id/1S_QEvwuA5fxYIBSwSiE3L8oOvREzRGi8/Final/ten_video/video_32.mp4: 640x640 1 person, 11.6ms
video 1/1 (2/300) /content/drive/.shor

In [ ]:
!python detect.py --source '/content/drive/MyDrive/DeepLearning/Final/ten_video/video_36.mp4' --weights runs/train/exp4/weights/last.pt --conf 0.25 --img-size 640 --save-txt --project '/content/drive/MyDrive/DeepLearning/Final/fine_tune_output' --name 'output_36'


detect: weights=['runs/train/exp4/weights/last.pt'], source=/content/drive/MyDrive/DeepLearning/Final/ten_video/video_36.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/DeepLearning/Final/fine_tune_output, name=output_36, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-150-gbe61a64 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/300) /content/drive/.shortcut-targets-by-id/1S_QEvwuA5fxYIBSwSiE3L8oOvREzRGi8/Final/ten_video/video_36.mp4: 640x640 2 persons, 11.6ms
video 1/1 (2/300) /content/drive/.sho

In [ ]:
!python detect.py --source '/content/drive/MyDrive/DeepLearning/Final/ten_video/video_80.mp4' --weights runs/train/exp4/weights/last.pt --conf 0.25 --img-size 640 --save-txt --project '/content/drive/MyDrive/DeepLearning/Final/fine_tune_output' --name 'output_80'


detect: weights=['runs/train/exp4/weights/last.pt'], source=/content/drive/MyDrive/DeepLearning/Final/ten_video/video_80.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/DeepLearning/Final/fine_tune_output, name=output_80, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-150-gbe61a64 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/300) /content/drive/.shortcut-targets-by-id/1S_QEvwuA5fxYIBSwSiE3L8oOvREzRGi8/Final/ten_video/video_80.mp4: 640x640 2 persons, 11.6ms
video 1/1 (2/300) /content/drive/.sho